In [13]:
import torch
import torch.nn.functional as F


In [3]:
tensor = torch.tensor([1.11, 2.22,3.33])

In [4]:
print(tensor)
print(f"size : {tensor.shape}")

tensor([1.1100, 2.2200, 3.3300])
size : torch.Size([3])


In [5]:
# Unsqueeze (-1)
tensor_usq = tensor.unsqueeze(-1)
print(tensor_usq)
print(f"size : {tensor_usq.shape}")

tensor([[1.1100],
        [2.2200],
        [3.3300]])
size : torch.Size([3, 1])


In [6]:

tensor_usq0 = tensor.unsqueeze(0)
print(tensor_usq0)

tensor([[1.1100, 2.2200, 3.3300]])


In [7]:
tensor_usq1 = tensor.unsqueeze(1)
print(tensor_usq1)

tensor([[1.1100],
        [2.2200],
        [3.3300]])


In [8]:
tensor_view = tensor.view(3,1,1)
print(tensor_view)

tensor([[[1.1100]],

        [[2.2200]],

        [[3.3300]]])


In [9]:
double_usq = tensor.unsqueeze(-1).unsqueeze(-1)
print(double_usq)

tensor([[[1.1100]],

        [[2.2200]],

        [[3.3300]]])


In [10]:
sift_feature = 5
cluster = 3
my_rand = torch.rand(cluster, sift_feature)

In [11]:
my_rand

tensor([[0.1871, 0.2464, 0.4505, 0.6111, 0.1410],
        [0.3883, 0.9948, 0.7917, 0.1360, 0.2118],
        [0.8362, 0.9717, 0.1571, 0.8393, 0.2109]])

In [12]:
mb = my_rand.unsqueeze(-1)
print(mb)
print(mb.size())

tensor([[[0.1871],
         [0.2464],
         [0.4505],
         [0.6111],
         [0.1410]],

        [[0.3883],
         [0.9948],
         [0.7917],
         [0.1360],
         [0.2118]],

        [[0.8362],
         [0.9717],
         [0.1571],
         [0.8393],
         [0.2109]]])
torch.Size([3, 5, 1])


In [13]:
mbb = my_rand.unsqueeze(-1).unsqueeze(-1)
print(mbb)
print(mbb.size())

tensor([[[[0.1871]],

         [[0.2464]],

         [[0.4505]],

         [[0.6111]],

         [[0.1410]]],


        [[[0.3883]],

         [[0.9948]],

         [[0.7917]],

         [[0.1360]],

         [[0.2118]]],


        [[[0.8362]],

         [[0.9717]],

         [[0.1571]],

         [[0.8393]],

         [[0.2109]]]])
torch.Size([3, 5, 1, 1])


In [14]:
import torchinfo
from torchvision.models import resnet18
import torch.nn as nn
import torch.nn.functional as F

In [15]:
encoder = resnet18(pretrained=True)
base_model = nn.Sequential(
    encoder.conv1,
    encoder.bn1,
    encoder.relu,
    encoder.maxpool,
    encoder.layer1,
    encoder.layer2,
    encoder.layer3,
    encoder.layer4,
)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
dim = list(base_model.parameters())[-1].shape # last channels (512)
print(dim)


torch.Size([512])


In [17]:
torchinfo.summary(base_model.cuda(), input_size=(1,3,128,128))

RuntimeError: No CUDA GPUs are available

In [ ]:
[1,512,4,4][0:2]

In [ ]:
new = torch.rand(1,512,4,4)
print(new.view(1,512,-1).shape)

In [ ]:
my_conv = nn.Conv2d(
            512, 
            6, 
            kernel_size=(1, 1), 
            bias=True
        )

In [ ]:
import torchinfo

In [ ]:
torchinfo.summary(my_conv.cuda(), input_size=(1,512,4,4))

In [ ]:
ss = torch.rand(1,6,4,4).view(1,6,-1)
print(ss)

In [ ]:
ss.shape

In [ ]:
ss.size(-1)

In [ ]:
F.softmax(ss, dim=0)

In [ ]:
sss = F.softmax(ss, dim=1)

In [ ]:
sss

In [ ]:
bb = torch.tensor([
    [[1,2,3]],
    [[4,5,6]]
],dtype=float)
print(bb)
print(bb.shape)

In [ ]:
F.softmax(ss, dim=1)

In [ ]:
0.0900+0.2447+0.6652

In [ ]:
class NetVLAD(nn.Module):
    """NetVLAD layer implementation"""

    def __init__(self, num_clusters=64, dim=128, 
                 normalize_input=True, vladv2=False):
        """
        Args:
            num_clusters : int
                The number of clusters
            dim : int
                Dimension of descriptors
            alpha : float
                Parameter of initialization. Larger value is harder assignment.
            normalize_input : bool
                If true, descriptor-wise L2 normalization is applied to input.
            vladv2 : bool
                If true, use vladv2 otherwise use vladv1
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = 0
        self.vladv2 = vladv2
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=vladv2)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))

    def init_params(self, clsts, traindescs):
        #TODO replace numpy ops with pytorch ops
        if self.vladv2 == False:
            clstsAssign = clsts / np.linalg.norm(clsts, axis=1, keepdims=True)
            dots = np.dot(clstsAssign, traindescs.T)
            dots.sort(0)
            dots = dots[::-1, :] # sort, descending

            self.alpha = (-np.log(0.01) / np.mean(dots[0,:] - dots[1,:])).item()
            self.centroids = nn.Parameter(torch.from_numpy(clsts))
            self.conv.weight = nn.Parameter(torch.from_numpy(self.alpha*clstsAssign).unsqueeze(2).unsqueeze(3))
            self.conv.bias = None
       

    def forward(self, x):
        N, C = x.shape[:2]

        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)  # across descriptor dim

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = x.view(N, C, -1)
        
        # calculate residuals to each clusters
        vlad = torch.zeros([N, self.num_clusters, C], dtype=x.dtype, layout=x.layout, device=x.device)
        for C in range(self.num_clusters): # slower than non-looped, but lower memory usage 
            residual = x_flatten.unsqueeze(0).permute(1, 0, 2, 3) - \
                    self.centroids[C:C+1, :].expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
            residual *= soft_assign[:,C:C+1,:].unsqueeze(2)
            vlad[:,C:C+1,:] = residual.sum(dim=-1)

        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize

        return vlad

In [ ]:
model_netvlad = NetVLAD(6,128,100,True)


In [ ]:
torchinfo.summary(model_netvlad, input_size=(1,128,4,4))

In [140]:
entries = 5
embedded_dim = 3
embeddings = torch.rand(entries, embedded_dim) * 100e-2

In [141]:
embeddings

tensor([[0.2165, 0.2350, 0.5292],
        [0.0958, 0.3472, 0.2017],
        [0.8312, 0.3126, 0.7385],
        [0.5124, 0.7426, 0.2795],
        [0.5861, 0.9412, 0.3736]])

In [142]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [143]:
def _pairwise_distance(x, squared=False, eps=1e-16):
    # Compute the 2D matrix of distances between all the embeddings.

    cor_mat = torch.matmul(x, x.t())
    print("Correlation Matrix : \n{}".format(cor_mat))
    norm_mat = cor_mat.diag()
    print("Norm Matrix : \n{}".format(norm_mat))
    print("NORM UNSQUEEZE(0) : \n{}".format(norm_mat.unsqueeze(0)))
    print("NORM UNSQUEEZE(1) : \n{}".format(norm_mat.unsqueeze(1)))
    
    print("2xCorMat : \n{}".format(2 * cor_mat))
    print("NormMat.UQ(1) Minus 2xCorMat : \n{}".format(norm_mat.unsqueeze(1) - 2 * cor_mat))
    distances = norm_mat.unsqueeze(1) - 2 * cor_mat + norm_mat.unsqueeze(0)
    print("Dist Matrix before relu : \n{}".format(distances))
    
    distances = F.relu(distances)
    print("Dist Matrix After relu : \n{}".format(distances))
    
    ccc = torch.sqrt(distances)
    
    if not squared:
        mask = torch.eq(distances, 0.0).float()
        print("Mask : \n{}".format(mask))
        
        distances = distances + mask * eps
        print("Distance After +Mask : \n{}".format(distances))
        
        distances = torch.sqrt(distances)
        distances = distances * (1.0 - mask)
        
        print("EQ : \n{}".format(torch.eq(distances, ccc)))

    return distances

In [144]:
_pairwise_distance(embeddings)

Correlation Matrix : 
tensor([[0.3821, 0.2090, 0.6442, 0.4333, 0.5457],
        [0.2090, 0.1704, 0.3371, 0.3633, 0.4583],
        [0.6442, 0.3371, 1.3340, 0.8644, 1.0573],
        [0.4333, 0.3633, 0.8644, 0.8921, 1.1036],
        [0.5457, 0.4583, 1.0573, 1.1036, 1.3689]])
Norm Matrix : 
tensor([0.3821, 0.1704, 1.3340, 0.8921, 1.3689])
NORM UNSQUEEZE(0) : 
tensor([[0.3821, 0.1704, 1.3340, 0.8921, 1.3689]])
NORM UNSQUEEZE(1) : 
tensor([[0.3821],
        [0.1704],
        [1.3340],
        [0.8921],
        [1.3689]])
2xCorMat : 
tensor([[0.7642, 0.4181, 1.2884, 0.8666, 1.0915],
        [0.4181, 0.3408, 0.6743, 0.7266, 0.9165],
        [1.2884, 0.6743, 2.6680, 1.7289, 2.1147],
        [0.8666, 0.7266, 1.7289, 1.7842, 2.2073],
        [1.0915, 0.9165, 2.1147, 2.2073, 2.7378]])
NormMat.UQ(1) Minus 2xCorMat : 
tensor([[-0.3821, -0.0360, -0.9063, -0.4845, -0.7094],
        [-0.2477, -0.1704, -0.5039, -0.5562, -0.7461],
        [ 0.0456,  0.6597, -1.3340, -0.3949, -0.7807],
        [ 0.0255,  

tensor([[0.0000, 0.3666, 0.6540, 0.6384, 0.8121],
        [0.3666, 0.0000, 0.9111, 0.5796, 0.7892],
        [0.6540, 0.9111, 0.0000, 0.7051, 0.7670],
        [0.6384, 0.5796, 0.7051, 0.0000, 0.2318],
        [0.8121, 0.7892, 0.7670, 0.2318, 0.0000]])

In [83]:
_pairwise_distance(embeddings)[0:1,]

Correlation Matrix : 
tensor([[1.0041, 0.3241, 0.6687, 0.6436, 1.1105],
        [0.3241, 0.9661, 0.3784, 0.6407, 0.8678],
        [0.6687, 0.3784, 0.4773, 0.5055, 0.8240],
        [0.6436, 0.6407, 0.5055, 0.6477, 1.0105],
        [1.1105, 0.8678, 0.8240, 1.0105, 1.6329]])
Norm Matrix : 
tensor([1.0041, 0.9661, 0.4773, 0.6477, 1.6329])
NORM UNSQUEEZE(1) : 
tensor([[1.0041],
        [0.9661],
        [0.4773],
        [0.6477],
        [1.6329]])
NORM UNSQUEEZE(0) : 
tensor([[1.0041, 0.9661, 0.4773, 0.6477, 1.6329]])
FACT CHECK tensor([1.0041, 0.9661, 0.4773, 0.6477, 1.6329])
Dist Matrix before relu : 
tensor([[0.0000, 1.3220, 0.1441, 0.3647, 0.4161],
        [1.3220, 0.0000, 0.6865, 0.3325, 0.8635],
        [0.1441, 0.6865, 0.0000, 0.1140, 0.4623],
        [0.3647, 0.3325, 0.1140, 0.0000, 0.2597],
        [0.4161, 0.8635, 0.4623, 0.2597, 0.0000]])
Dist Matrix After relu : 
tensor([[0.0000, 1.3220, 0.1441, 0.3647, 0.4161],
        [1.3220, 0.0000, 0.6865, 0.3325, 0.8635],
        [0.1441

tensor([[0.0000, 1.1498, 0.3796, 0.6039, 0.6451]])

In [46]:
import numpy as np

In [64]:
a = np.array(embeddings[0].cpu())
b = np.array(embeddings[1].cpu())
c = a-b

In [65]:
c

array([ 0.89680624, -0.7028398 , -0.15422863], dtype=float32)

In [66]:
np.sum(c)

0.03973782

In [67]:
np.sum(np.abs(c))

1.7538748

In [68]:
np.power(c,2)

array([0.80426145, 0.49398378, 0.02378647], dtype=float32)

In [70]:
np.sqrt(np.sum(np.square(c)))

1.1497964

In [96]:
a1 = torch.rand(5,1)
a1

tensor([[0.5145],
        [0.8516],
        [0.9675],
        [0.5447],
        [0.2865]])

In [97]:
a2 = torch.rand(5,5)
a2

tensor([[0.0832, 0.7014, 0.5694, 0.4796, 0.0708],
        [0.0416, 0.1154, 0.3853, 0.8826, 0.6110],
        [0.6124, 0.2818, 0.1288, 0.9304, 0.4217],
        [0.2345, 0.7804, 0.1424, 0.5926, 0.3617],
        [0.9637, 0.0379, 0.8967, 0.9954, 0.9833]])

In [98]:
a1+a2

tensor([[0.5977, 1.2160, 1.0840, 0.9941, 0.5853],
        [0.8932, 0.9669, 1.2368, 1.7342, 1.4625],
        [1.5799, 1.2492, 1.0962, 1.8978, 1.3891],
        [0.7791, 1.3251, 0.6870, 1.1373, 0.9063],
        [1.2502, 0.3245, 1.1832, 1.2819, 1.2698]])

In [102]:
a2

tensor([[0.0832, 0.7014, 0.5694, 0.4796, 0.0708],
        [0.0416, 0.1154, 0.3853, 0.8826, 0.6110],
        [0.6124, 0.2818, 0.1288, 0.9304, 0.4217],
        [0.2345, 0.7804, 0.1424, 0.5926, 0.3617],
        [0.9637, 0.0379, 0.8967, 0.9954, 0.9833]])

In [111]:
a3 = a2 - a2.diag()
a3 = F.relu(a3)
a3

tensor([[0.0000, 0.5861, 0.4407, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2565, 0.2900, 0.0000],
        [0.5293, 0.1664, 0.0000, 0.3377, 0.0000],
        [0.1513, 0.6650, 0.0136, 0.0000, 0.0000],
        [0.8805, 0.0000, 0.7679, 0.4027, 0.0000]])

In [112]:
mask = torch.eq(a3, 0.0).float()
mask

tensor([[1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1.],
        [0., 1., 0., 0., 1.]])

In [113]:
a4 = a3 + mask*1e-16
a4

tensor([[1.0000e-16, 5.8606e-01, 4.4066e-01, 1.0000e-16, 1.0000e-16],
        [1.0000e-16, 1.0000e-16, 2.5648e-01, 2.8995e-01, 1.0000e-16],
        [5.2926e-01, 1.6640e-01, 1.0000e-16, 3.3774e-01, 1.0000e-16],
        [1.5129e-01, 6.6504e-01, 1.3589e-02, 1.0000e-16, 1.0000e-16],
        [8.8054e-01, 1.0000e-16, 7.6794e-01, 4.0273e-01, 1.0000e-16]])

In [114]:
a5 = torch.sqrt(a4)
a5

tensor([[1.0000e-08, 7.6555e-01, 6.6382e-01, 1.0000e-08, 1.0000e-08],
        [1.0000e-08, 1.0000e-08, 5.0644e-01, 5.3847e-01, 1.0000e-08],
        [7.2750e-01, 4.0792e-01, 1.0000e-08, 5.8116e-01, 1.0000e-08],
        [3.8895e-01, 8.1550e-01, 1.1657e-01, 1.0000e-08, 1.0000e-08],
        [9.3837e-01, 1.0000e-08, 8.7632e-01, 6.3461e-01, 1.0000e-08]])

In [116]:
(1.0 - mask)

tensor([[0., 1., 1., 0., 0.],
        [0., 0., 1., 1., 0.],
        [1., 1., 0., 1., 0.],
        [1., 1., 1., 0., 0.],
        [1., 0., 1., 1., 0.]])

In [115]:
a6 = a5 * (1.0 - mask)
a6

tensor([[0.0000, 0.7655, 0.6638, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5064, 0.5385, 0.0000],
        [0.7275, 0.4079, 0.0000, 0.5812, 0.0000],
        [0.3890, 0.8155, 0.1166, 0.0000, 0.0000],
        [0.9384, 0.0000, 0.8763, 0.6346, 0.0000]])

In [119]:
ss = torch.zeros(5,5)
ss

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [120]:
torch.sqrt(ss)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [147]:
torch.sqrt(torch.zeros(1))

tensor([0.])

In [148]:
labels = torch.tensor([1,1,2,2,3,3,3,1,1])
labels

tensor([1, 1, 2, 2, 3, 3, 3, 1, 1])

In [149]:
def _get_anchor_positive_triplet_mask(labels):
    """
    Given labels of size (batch_size, ) 
    Return a 2D Mask : Where mask[Anchor, Positive] is True
    iff Anchor and Positive 
    - Distinct 
    - Same Label
    
    Mask will omitted the self index (result 0)
    label [1,1,2,2,3,1] 
    for item idx 0 will return [X(zero),1,0,0,0,1]
    """
    # Device Select
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Negative Mask
    indices_not_equal = torch.eye(labels.shape[0]).to(device).byte() ^ 1

    # Check if labels[i] == labels[j]
    labels_equal = torch.unsqueeze(labels, 0) == torch.unsqueeze(labels, 1)
    
    mask = indices_not_equal * labels_equal

    return mask

In [150]:
_get_anchor_positive_triplet_mask(labels)

tensor([[0, 1, 0, 0, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=torch.uint8)

In [151]:
def _get_anchor_negative_triplet_mask(labels):
    # Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.

    # Check if labels[i] != labels[k]
    labels_equal = torch.unsqueeze(labels, 0) == torch.unsqueeze(labels, 1)
    mask = labels_equal ^ 1

    return mask

In [152]:
_get_anchor_negative_triplet_mask(labels)

tensor([[0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0]])